In [1]:
import json
import pickle
from flask import  Flask, request, jsonify, render_template
import os
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV

In [2]:
# Iniciamos nuestra API
app = Flask('Predictor Indice de Incobrabilidad', template_folder='Plantillas')

In [3]:
@app.route('/')
def home():
    return render_template('index.html')
# Dado que en este endopint no vamos a guardar nada en el servidor, sino recibir información, usamos 
# el método GET. 

@app.route("/predict",methods=['POST'])
def predict():
    
    # Usamos request.args para tomar las query que le pasamos a la URL
    badlar=float(request.form['badlar'])
    tc=float(request.form['tc'])
    ipc=float(request.form['ipc'])
    pbi_real=float(request.form['pbi_real'])
    pbi_nominal=float(request.form['pbi_nominal'])
    tasa_desempleo=float(request.form['tasa_desempleo'])
   
    # A partir de las features recibidas, reformateamos los datos para que puedan ser ingresados al modelo.
    
    features_dict = {'badlar':[badlar],
                     'tc':[tc],
                     'ipc':[ipc],
                     'pbi_real':[pbi_real],
                     'pbi_nominal':[pbi_nominal],
                     'tasa_desempleo':[tasa_desempleo],
                     'badlar_2':[badlar**2],
                     'tc_2':[tc**2],
                     'ipc_2':[ipc**2],
                     'pbi_real_2':[pbi_real**2],
                     'pbi_nominal_2':[pbi_nominal**2],
                     'tasa_desempleo_2':[tasa_desempleo**2]
                   }
    
    features_df = pd.DataFrame(features_dict)
    
    print(features_df)
    
    # Levantamos el modelo y el escalador que ya tenemos entrenado
    
    with open('ridge_model.pkl', 'rb') as modelo_pkl:
        modelo_load = pickle.load(modelo_pkl)
        
    with open('scaler.pkl', 'rb') as scaler_pkl:
        scaler_load = pickle.load(scaler_pkl)
        
    # re-escalamos los datos con el escalador entrenado (tengan en cuenta que el escalador va a estar
    # esperando una estructura como un DataFrame)
    scaled_case=scaler_load.transform(features_df)
    
    print("Datos escalados: ",scaled_case)
    
    # realizamos la prediccion
    prediccion=modelo_load.predict(scaled_case).round(2)


    return render_template('index.html', prediction_text=f'La tasa de incobrabilidad estimada es {prediccion}')

In [ ]:
# ejecutar esta celda antes de entrar en la dirección): http://localhost:5000/
app.run(host='0.0.0.0')

 * Serving Flask app "Predictor Indice de Incobrabilidad" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [31/Mar/2021 12:06:48] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/Mar/2021 12:08:09] "POST /predict HTTP/1.1" 200 -


   badlar      tc     ipc   pbi_real  pbi_nominal  tasa_desempleo   badlar_2  \
0   36.01  126.13  559.55  691656.54  43643792.92            12.2  1296.7201   

         tc_2        ipc_2    pbi_real_2  pbi_nominal_2  tasa_desempleo_2  
0  15908.7769  313096.2025  4.783888e+11   1.904781e+15            148.84  
Datos escalados:  [[ 1.43315019  5.43608579  5.10537723  0.47849229  4.72595607  2.11665575
   1.21273496 10.33554386  9.54923446  0.45228374  8.98430892  2.27098498]]
